# Pergunta 3 - Quantas fotos são adicionadas e como se distribuem por visita ou projeto?

contagem das fotos

In [0]:
%sql
SELECT COUNT(*) AS total_fotos
FROM gold.fato_foto_adicionada;


Quantidade de foto por visita

In [0]:
%sql
SELECT 
    v.nome_visita,
    COUNT(*) AS qtd_fotos
FROM gold.fato_foto_adicionada ff
JOIN gold.dim_visita v ON v.visita_id = ff.visita_id
GROUP BY v.nome_visita
ORDER BY qtd_fotos DESC;


quantidade de fotos por projeto

In [0]:
%sql
SELECT 
    p.nome_projeto,
    COUNT(*) AS qtd_fotos
FROM gold.fato_foto_adicionada ff
JOIN gold.dim_projeto p ON p.projeto_id = ff.projeto_id
GROUP BY p.nome_projeto
ORDER BY qtd_fotos DESC;


quem mais sobe fotos?

In [0]:
%sql
SELECT 
    u.email,
    COUNT(*) AS fotos_tiradas
FROM gold.fato_foto_adicionada ff
JOIN gold.dim_usuario u ON u.usuario_id = ff.usuario_id
GROUP BY u.email
ORDER BY fotos_tiradas DESC
LIMIT 20;


 Usuarios com mais fotos fazem mais visitas?

In [0]:
%sql
SELECT
    u.email,
    COUNT(DISTINCT fv.id_fato_visita) AS visitas,
    COUNT(DISTINCT ff.id_fato_foto) AS fotos
FROM gold.dim_usuario u
LEFT JOIN gold.fato_visita_criada fv ON fv.usuario_id = u.usuario_id
LEFT JOIN gold.fato_foto_adicionada ff ON ff.usuario_id = u.usuario_id
GROUP BY u.email
ORDER BY fotos DESC
limit 30;


Quantas fotos tem marcador e quais clientes usaram

In [0]:
%sql
SELECT 
    foto_tem_marcador,
    COUNT(*) AS total
FROM gold.fato_foto_adicionada
GROUP BY foto_tem_marcador;


In [0]:
%sql
SELECT 
    u.email AS cliente_email,
    COUNT(*) AS fotos_marcadas
FROM gold.fato_foto_adicionada ff
JOIN gold.dim_usuario u
    ON ff.usuario_id = u.usuario_id
WHERE ff.foto_cliente = TRUE
  AND ff.foto_tem_marcador = TRUE
GROUP BY u.email
ORDER BY fotos_marcadas DESC;


média de fotos por visita e por projeto

In [0]:
%sql
SELECT 
    ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT visita_id), 2) AS media_fotos_por_visita
FROM gold.fato_foto_adicionada;


In [0]:
%sql
SELECT 
    ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT projeto_id), 2) AS media_fotos_por_projeto
FROM gold.fato_foto_adicionada;


histograma de fotos por visita

In [0]:
%sql
WITH fotos_por_visita AS (
    SELECT 
        visita_id,
        COUNT(*) AS qtd_fotos
    FROM gold.fato_foto_adicionada
    GROUP BY visita_id
)

SELECT 
    CASE 
        WHEN qtd_fotos BETWEEN 0 AND 5 THEN '0 a 5 fotos'
        WHEN qtd_fotos BETWEEN 6 AND 11 THEN '6 a 11 fotos'
        WHEN qtd_fotos BETWEEN 12 AND 20 THEN '12 a 20 fotos'
        WHEN qtd_fotos BETWEEN 21 AND 35 THEN '21 a 35 fotos'
        WHEN qtd_fotos BETWEEN 36 AND 50 THEN '36 a 50 fotos'
        WHEN qtd_fotos > 50 THEN '50+ fotos'
    END AS faixa,
    COUNT(*) AS quantidade_de_visitas
FROM fotos_por_visita
GROUP BY faixa
ORDER BY 
    CASE faixa
        WHEN '0 a 5 fotos' THEN 1
        WHEN '6 a 11 fotos' THEN 2
        WHEN '12 a 20 fotos' THEN 3
        WHEN '21 a 35 fotos' THEN 4
        WHEN '36 a 50 fotos' THEN 5
        WHEN '50+ fotos' THEN 6
    END;
